In [609]:
import pandas as pd
import sqlite3
import numpy as np

In [610]:
df = pd.read_excel(r'C:\Users\srija\OneDrive\Desktop\merged.xlsx')

In [611]:
df.head()

,Vendor,VendorName,Invoice,GLs,category,Costctr,Amount,On
0,TFET,abcde,324234,1,Vendor,a1,455,2023-10-01
1,TFET,abcde,24334,1,Vendor,a1,455,2023-10-01
2,TFET,abcde,24124,3,Vendor,a1,455,2023-10-01
3,1234,emper,345646,5,Employee,b1,987,2023-10-01
4,1234,emper,34235,4,Employee,b2,654,2023-10-01


In [612]:
df1 = pd.read_excel(r'C:\Users\srija\OneDrive\Desktop\costctr.xlsx', header = 1)
df2 = pd.read_excel(r'C:\Users\srija\OneDrive\Desktop\GLNAME.xlsx', header = 1)
df1.head()

,Unnamed: 0,Costct,Cname
0,NaN,a1,area1
1,NaN,a2,area2
2,NaN,a3,area3
3,NaN,b1,book1
4,NaN,b2,book2


In [613]:
df1.rename(columns = {'Costct': 'Costctr'}, inplace = True)

In [614]:
df2.rename(columns = {'Glaccount': 'GLs'}, inplace = True)

In [615]:
df2.head()

,Unnamed: 0,GLs,Gname
0,NaN,1,jhjh11
1,NaN,2,jhghftu22
2,NaN,3,jhhgf33
3,NaN,4,xdrg44
4,NaN,5,zzz55


In [616]:
df_gl = df2[['GLs', 'Gname']]

In [617]:
df_ctr= df1[['Costctr', 'Cname']]

In [618]:
df_gl

,GLs,Gname
0,1,jhjh11
1,2,jhghftu22
2,3,jhhgf33
3,4,xdrg44
4,5,zzz55
5,c1,car1
6,6,hgtt6
7,dsf,car3


In [619]:
costctr_to_cname = dict(zip(df_ctr['Costctr'], df_ctr['Cname']))
df['Cname'] = df['Costctr'].map(costctr_to_cname)

In [620]:
costctr_to_cname = dict(zip(df_gl['GLs'], df_gl['Gname']))
df['Gname'] = df['GLs'].map(costctr_to_cname)

In [621]:
df['On'] = pd.to_datetime(df['On'])

In [622]:
df.head()

,Vendor,VendorName,Invoice,GLs,category,Costctr,Amount,On,Cname,Gname
0,TFET,abcde,324234,1,Vendor,a1,455,2023-10-01,area1,jhjh11
1,TFET,abcde,24334,1,Vendor,a1,455,2023-10-01,area1,jhjh11
2,TFET,abcde,24124,3,Vendor,a1,455,2023-10-01,area1,jhhgf33
3,1234,emper,345646,5,Employee,b1,987,2023-10-01,book1,zzz55
4,1234,emper,34235,4,Employee,b2,654,2023-10-01,book2,xdrg44


In [623]:
df['year'] = df['On'].dt.year

In [624]:
df['On'] = df['On'].dt.date

In [625]:
df['Cummulative_transactions'] = len(df)
df['Cummulative_transactions/category'] = df.groupby('category')['category'].transform('count')
df['overall_transactions/year'] = df.groupby('year')['year'].transform('count')
df['overall_transactions/category/year'] = df.groupby(['category', 'year'])['category'].transform('count')

In [640]:
df['cumulative_Alloted_Amount'] = df['Amount'].sum()

In [627]:
df['cumulative_Alloted_Amount\Category'] = df.groupby('category')['Amount'].transform('sum')

In [628]:
df['Yearly_Alloted_Amount\Category'] = df.groupby(['category', 'year'])['Amount'].transform('sum')

In [629]:
yearly_total = df.groupby('year')['Amount'].sum().reset_index()
yearly_total.rename(columns={'Amount': 'Total_Alloted_Amount/year'}, inplace=True)
df = pd.merge(df, yearly_total, on='year', how='left')

In [630]:
df['cumulative_Transations/Vendor'] = df.groupby('Vendor')['Vendor'].transform('count')

In [631]:
df['Transations/year/Vendor'] = df.groupby(['year', 'Vendor'])['Vendor'].transform('count')

In [632]:
df['Cumulative_Amount_used'] = df.groupby(['Vendor', 'category'])['Amount'].transform('sum')

In [633]:
df['Amount_used/Year'] = df.groupby(['Vendor', 'year', 'category'])['Amount'].transform('sum')

In [634]:
df['Cumulative_percentageamount_used'] = (df['Cumulative_Amount_used'] / df['cumulative_Alloted_Amount']) * 100

In [635]:
df['percentage_amount_used_per_year'] = (df['Amount_used/Year'] / df['Total_Alloted_Amount/year']) * 100

In [636]:
df['total_percentage_of_amount/category_used'] = (df['Cumulative_Amount_used'] / df['cumulative_Alloted_Amount\Category']) * 100

In [637]:
df['percentage_of_amount/category_used/year'] = (df['Amount_used/Year'] / df['Yearly_Alloted_Amount\Category']) * 100

In [638]:
df['Cumulative_percentransations_made'] = (df['cumulative_Transations/Vendor'] / df['Cummulative_transactions']) * 100
df['Cumulative_percentransations_made/category'] = (df['cumulative_Transations/Vendor'] / df['Cummulative_transactions/category']) * 100
df['percentransations_made/category/year'] = (df['Transations/year/Vendor'] / df['overall_transactions/category/year']) * 100
df['percentransations_made/year'] = (df['Transations/year/Vendor'] / df['overall_transactions/year']) * 100

In [639]:
df.head()

,Vendor,VendorName,Invoice,GLs,category,Costctr,Amount,On,Cname,Gname,year,Cummulative_transactions,Cummulative_transactions/category,overall_transactions/year,overall_transactions/category/year,cumulative_Alloted_Amount,cumulative_Alloted_Amount\Category,Yearly_Alloted_Amount\Category,Total_Alloted_Amount/year,cumulative_Transations/Vendor,Transations/year/Vendor,Cumulative_Amount_used,Amount_used/Year,Cumulative_percentageamount_used,percentage_amount_used_per_year,total_percentage_of_amount/category_used,percentage_of_amount/category_used/year,Cumulative_percentransations_made,Cumulative_percentransations_made/category,percentransations_made/category/year,percentransations_made/year
0,TFET,abcde,324234,1,Vendor,a1,455,2023-10-01,area1,jhjh11,2023,48,15,14,4,17818667,7686502,7352,3082553,8,3,3640,1365,0.020428,0.044281,0.047356,18.566376,16.666667,53.333333,75.000000,21.428571
1,TFET,abcde,24334,1,Vendor,a1,455,2023-10-01,area1,jhjh11,2023,48,15,14,4,17818667,7686502,7352,3082553,8,3,3640,1365,0.020428,0.044281,0.047356,18.566376,16.666667,53.333333,75.000000,21.428571
2,TFET,abcde,24124,3,Vendor,a1,455,2023-10-01,area1,jhhgf33,2023,48,15,14,4,17818667,7686502,7352,3082553,8,3,3640,1365,0.020428,0.044281,0.047356,18.566376,16.666667,53.333333,75.000000,21.428571
3,1234,emper,345646,5,Employee,b1,987,2023-10-01,book1,zzz55,2023,48,12,14,3,17818667,1041214,44884,3082553,7,2,769455,1641,4.318252,0.053235,73.899794,3.656091,14.583333,58.333333,66.666667,14.285714
4,1234,emper,34235,4,Employee,b2,654,2023-10-01,book2,xdrg44,2023,48,12,14,3,17818667,1041214,44884,3082553,7,2,769455,1641,4.318252,0.053235,73.899794,3.656091,14.583333,58.333333,66.666667,14.285714


In [602]:
df = df.drop_duplicates(subset=['Vendor', 'year'])
df.reset_index(drop=True, inplace=True)

In [603]:
df['Vendor'] = df['Vendor'].astype(str)
df_sorted = df.sort_values(by='Vendor',ascending = False)
df_sorted.reset_index(drop=True, inplace=True)

In [604]:
pd.set_option('display.max_columns', None)

In [605]:
df_sorted.tail()

,Vendor,VendorName,Invoice,GLs,category,Costctr,Amount,On,Cname,Gname,year,Cummulative_transactions,Cummulative_transactions/category,overall_transactions/year,overall_transactions/category/year,cumulative_Alloted_Amount,cumulative_Alloted_Amount\Category,Yearly_Alloted_Amount\Category,Total_Alloted_Amount/year,cumulative_Transations/Vendor,Transations/year/Vendor,Cumulative_Amount_used,Amount_used/Year,Cumulative_percentageamount_used,percentage_amount_used_per_year,total_percentage_of_amount/category_used,percentage_of_amount/category_used/year
25,110,AAB,DJSBSDJFN,2,others,c2,7532,2023-10-01,car2,jhghftu22,2023,48,12,14,4,17818667,3244092,1081364,3082553,6,2,52185,17395,0.292867,0.564305,1.608617,1.608617
26,110,AAB,DJSBSDJFN,2,others,c2,7532,2021-10-01,car2,jhghftu22,2021,48,12,14,4,17818667,3244092,1081364,11501304,6,2,52185,17395,0.292867,0.151244,1.608617,1.608617
27,100,HEGF,SDLKGJNJSDK,1,others,c1,77543,2023-10-01,car1,jhjh11,2023,48,12,14,4,17818667,3244092,1081364,3082553,3,1,232629,77543,1.305535,2.515545,7.170851,7.170851
28,100,HEGF,SDLKGJNJSDK,1,others,c1,77543,2021-10-01,car1,jhjh11,2021,48,12,14,4,17818667,3244092,1081364,11501304,3,1,232629,77543,1.305535,0.674211,7.170851,7.170851
29,100,HEGF,SDLKGJNJSDK,1,others,c1,77543,2022-10-01,car1,jhjh11,2022,48,12,20,4,17818667,3244092,1081364,3234810,3,1,232629,77543,1.305535,2.397142,7.170851,7.170851


In [606]:
create a python html code that takes a xcel file as input and when clicked on analysze button it should perform df = pd.read_excel(inputexcel)
df1 = pd.read_excel(r'costctr.xlsx', header = 1)
df2 = pd.read_excel(r'GLNAME.xlsx', header = 1)
df1.rename(columns = {'Costct': 'Costctr'}, inplace = True)
df2.rename(columns = {'Glaccount': 'GLs'}, inplace = True)
df_gl = df2[['GLs', 'Gname']]
df_ctr= df1[['Costctr', 'Cname']]
costctr_to_cname = dict(zip(df_ctr['Costctr'], df_ctr['Cname']))
df['Cname'] = df['Costctr'].map(costctr_to_cname)
costctr_to_cname = dict(zip(df_gl['GLs'], df_gl['Gname']))
df['Gname'] = df['GLs'].map(costctr_to_cname)
df['On'] = pd.to_datetime(df['On'])
df['year'] = df['On'].dt.year
df['On'] = df['On'].dt.date
df['cumulative_Alloted_Amount'] = df['Amount'].sum()
df['cumulative_Alloted_Amount\Category'] = df.groupby('category')['Amount'].transform('sum')
df['Yearly_Alloted_Amount\Category'] = df.groupby(['category', 'year'])['Amount'].transform('sum')
yearly_total = df.groupby('year')['Amount'].sum().reset_index()
yearly_total.rename(columns={'Amount': 'Total_Alloted_Amount/year'}, inplace=True)
df = pd.merge(df, yearly_total, on='year', how='left')
df['cumulative_Transations'] = df.groupby('Vendor')['Vendor'].transform('count')
df['Transations/year'] = df.groupby(['year', 'Vendor'])['Vendor'].transform('count')
df['Cumulative_Amount_used'] = df.groupby(['Vendor', 'category'])['Amount'].transform('sum')
df['Amount_used/Year'] = df.groupby(['Vendor', 'year', 'category'])['Amount'].transform('sum')
df['Cumulative_percentageamount_used'] = (df['Cumulative_Amount_used'] / df['cumulative_Alloted_Amount']) * 100
df['percentage_amount_used_per_year'] = (df['Amount_used/Year'] / df['Total_Alloted_Amount/year']) * 100
df['total_percentage_of_amount/category_used'] = (df['Cumulative_Amount_used'] / df['cumulative_Alloted_Amount\Category']) * 100
df['percentage_of_amount/category_used/year'] = (df['Amount_used/Year'] / df['Yearly_Alloted_Amount\Category']) * 100
df = df.drop_duplicates(subset=['Vendor', 'year'])
df.reset_index(drop=True, inplace=True)
df['Vendor'] = df['Vendor'].astype(str)
df_sorted = df.sort_values(by='Vendor',ascending = False)
df_sorted.reset_index(drop=True, inplace=True)
df_sorted

SyntaxError: invalid syntax (1458260660.py, line 1)

In [607]:

import streamlit as st
import pandas as pd
import base64
import os

# Define allowed file extensions
ALLOWED_EXTENSIONS = {'xlsx'}


# Function to check if file extension is allowed
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


# Function to perform analysis on Excel file
def analyze_excel(file):
    df = pd.read_excel(file)
    df1 = pd.read_excel(r"C:\Users\srija\OneDrive\Desktop\nonPO\costctr.xlsx", header=1)
    df2 = pd.read_excel(r"C:\Users\srija\OneDrive\Desktop\nonPO\GLNAME.xlsx", header=1)
    df1.rename(columns={'Costct': 'Costctr'}, inplace=True)
    df2.rename(columns={'Glaccount': 'GLs'}, inplace=True)
    df_gl = df2[['GLs', 'Gname']]
    df_ctr = df1[['Costctr', 'Cname']]
    costctr_to_cname = dict(zip(df_ctr['Costctr'], df_ctr['Cname']))
    df['Cname'] = df['Costctr'].map(costctr_to_cname)
    costctr_to_cname = dict(zip(df_gl['GLs'], df_gl['Gname']))
    df['Gname'] = df['GLs'].map(costctr_to_cname)
    df['On'] = pd.to_datetime(df['On'])
    df['year'] = df['On'].dt.year
    df['On'] = df['On'].dt.date
    df['cumulative_Alloted_Amount'] = df['Amount'].sum()
    df['cumulative_Alloted_Amount\Category'] = df.groupby('category')['Amount'].transform('sum')
    df['Yearly_Alloted_Amount\Category'] = df.groupby(['category', 'year'])['Amount'].transform('sum')
    yearly_total = df.groupby('year')['Amount'].sum().reset_index()
    yearly_total.rename(columns={'Amount': 'Total_Alloted_Amount/year'}, inplace=True)
    df = pd.merge(df, yearly_total, on='year', how='left')
    df['cumulative_Transations'] = df.groupby('Vendor')['Vendor'].transform('count')
    df['Transations/year'] = df.groupby(['year', 'Vendor'])['Vendor'].transform('count')
    df['Cumulative_Amount_used'] = df.groupby(['Vendor', 'category'])['Amount'].transform('sum')
    df['Amount_used/Year'] = df.groupby(['Vendor', 'year', 'category'])['Amount'].transform('sum')
    df['Cumulative_percentageamount_used'] = (df['Cumulative_Amount_used'] / df['cumulative_Alloted_Amount']) * 100
    df['percentage_amount_used_per_year'] = (df['Amount_used/Year'] / df['Total_Alloted_Amount/year']) * 100
    df['total_percentage_of_amount/category_used'] = (df['Cumulative_Amount_used'] / df[
        'cumulative_Alloted_Amount\Category']) * 100
    df['percentage_of_amount/category_used/year'] = (df['Amount_used/Year'] / df[
        'Yearly_Alloted_Amount\Category']) * 100
    df = df.drop_duplicates(subset=['Vendor', 'year'])
    df.reset_index(drop=True, inplace=True)
    df['Vendor'] = df['Vendor'].astype(str)
    df_sorted = df.sort_values(by='Vendor', ascending=False)  # Showing only the first 25 rows
    df_sorted.reset_index(drop=True, inplace=True)

    # Storing the resultant DataFrame to an Excel file
    df_sorted.to_excel("analysis_result.xlsx", index=False)

    return df_sorted


# Main Streamlit app
def main():
    st.title('Excel Analyzer')

    # Divide the page into 3 columns
    col1, col2, col3 = st.columns(3)

    # Placeholder for file uploader
    with col1:
        upload_placeholder = st.empty()

    # Placeholder for analyze button
    with col2:
        analyze_placeholder = st.empty()

    # File uploader
    uploaded_file = upload_placeholder.file_uploader("Upload Excel file", type=["xlsx"])

    if uploaded_file is not None:
        if allowed_file(uploaded_file.name):
            df = analyze_excel(uploaded_file)
            analyze_placeholder.empty()  # Hide analyze button
            st.write("### Analysis Result")

            # Dropdowns for selecting year, category, and graph type
            with col1:
                year_options = list(df['year'].unique()) + ['All']
                selected_year = st.selectbox("Select Year", year_options)

            with col2:
                category_options = list(df['category'].unique()) + ['All']
                selected_category = st.selectbox("Select Category", category_options)

            with col3:
                graph_options = ['Payment Trend', 'Transaction Trend']
                selected_graph = st.selectbox("Select Graph Type", graph_options)

            # Filtering and sorting the DataFrame based on selected options
            if selected_year != 'All':
                df = df[df['year'] == selected_year]
                # If a specific year is selected, sort by Amount_used/Year
                df = df.sort_values(by='Amount_used/Year', ascending=False)
            else:
                # If 'All' is selected, sort by Cumulative_Amount_used
                df = df.sort_values(by='Cumulative_Amount_used', ascending=False)

            if selected_category != 'All':
                df = df[df['category'] == selected_category]

            df.reset_index(drop=True, inplace=True)  # Reset index after filtering and sorting

            st.write(df)
            st.markdown(get_binary_file_downloader_html("analysis_result.xlsx", 'Download Excel'),
                        unsafe_allow_html=True)
            upload_placeholder.empty()  # Hide file uploader
        else:
            st.write("Invalid file type. Please upload a .xlsx file.")


# Function to create a download link for files
def get_binary_file_downloader_html(bin_file, file_label='File'):
    with open(bin_file, 'rb') as f:
        data = f.read()
    bin_str = base64.b64encode(data).decode()
    href = f'<a href="data:application/octet-stream;base64,{bin_str}" download="{os.path.basename(bin_file)}">Download {file_label}</a>'
    return href


if __name__ == "__main__":
    main()
again devide the page into 2 coloumns in one coloumn df should ve ther and in other the line graph should be there if (year,category = all and graph type = Payement trend Plot a line chart for Cumulative_percentageamount_used and Vendor if graph type = Transactions trend plot for 

SyntaxError: invalid syntax (615245438.py, line 129)